In [1]:
import pandas as pd
import numpy as np
import yfinance as yahooFinance
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import copy
import statsmodels.api as sm

In [2]:
dfPrices = pd.read_csv("PricesThru2022.csv")

In [3]:
display(dfPrices.head(),dfPrices.tail())

,Unnamed: 0,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
0,1999-12-31,0.917969,3.806250,36.820244,0.6567,798.154631,1.0062,396.794861,288.00,41.15625,5.380,146.8750
1,2000-01-03,0.999442,4.468750,35.498489,0.6585,793.868670,1.0243,384.615387,289.00,43.50000,5.405,145.4375
2,2000-01-04,0.915179,4.096875,33.421452,0.6550,796.761677,1.0296,369.230774,282.25,41.46875,5.325,139.7500
3,2000-01-05,0.928571,3.487500,33.940708,0.6579,793.118215,1.0321,368.589752,280.00,41.81250,5.145,140.0000
4,2000-01-06,0.848214,3.278125,33.043808,0.6533,795.091984,1.0328,373.517639,281.10,39.37500,5.120,137.7500


,Unnamed: 0,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
5699,2022-10-25,152.339996,120.599998,17.690001,0.6394,844.938544,0.9966,73.000000,1653.17,27.410000,19.3473,384.920013
5700,2022-10-26,149.350006,115.660004,18.139999,0.6497,849.013953,1.0081,75.459999,1664.57,27.209999,19.5815,382.019989
5701,2022-10-27,144.800003,110.959999,18.030001,0.6452,853.391436,0.9964,76.000000,1663.31,26.270000,19.6014,379.980011
5702,2022-10-28,155.740005,103.410004,18.480000,0.6411,851.238239,0.9965,78.330002,1644.86,29.070000,19.2595,389.019989
5703,2022-10-31,153.339996,102.440002,18.230000,0.6399,847.557048,0.9882,77.809998,1633.56,28.430000,19.1645,386.209991


In [4]:
display(dfPrices.info(),dfPrices.isnull().sum(),dfPrices.index)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5704 entries, 0 to 5703
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  5704 non-null   object 
 1   AAPL        5704 non-null   float64
 2   AMZN        5704 non-null   float64
 3   ATT         5704 non-null   float64
 4   AUD         5704 non-null   float64
 5   BOND        5704 non-null   float64
 6   EUR         5704 non-null   float64
 7   GE          5704 non-null   float64
 8   GOLD        5704 non-null   float64
 9   INTC        5704 non-null   float64
 10  SILVER      5704 non-null   float64
 11  SPY         5704 non-null   float64
dtypes: float64(11), object(1)
memory usage: 534.9+ KB


None

Unnamed: 0    0
AAPL          0
AMZN          0
ATT           0
AUD           0
BOND          0
EUR           0
GE            0
GOLD          0
INTC          0
SILVER        0
SPY           0
dtype: int64

RangeIndex(start=0, stop=5704, step=1)

## Q2: Create Portfolio:

In [5]:
def percent_function (dF, pct_format='{:.4%}', nan='NaN', indexSlice=pd.IndexSlice[:,:]):
    display(dF.style.format(pct_format, na_rep=nan, subset=indexSlice))

In [6]:
# calculate (and print) the summary statistics table
def calcSummaryStatistics(df, annualization_factor=260, pct_format='{:.2%}'):
    cumRtns = df.cumsum()
    summary_stats = pd.DataFrame([annualization_factor*df.mean(),
                                  np.sqrt(annualization_factor)*df.std(),
                                  np.sqrt(annualization_factor) * df.mean() / df.std(),
                                  (cumRtns - np.maximum.accumulate(cumRtns)).min(axis=0),
                                  (df >= 0).sum() / df.shape[0]
                                 ],
                                 index=['Average Annual Return', 'Annual Risk', 'Sharpe Ratio', 'maxDD', 'Success Ratio']
                                )
    percent_function(summary_stats, pct_format=pct_format, indexSlice=pd.IndexSlice[['Average Annual Return', 'Annual Risk', 'maxDD', 'Success Ratio'], :])
#     return summary_stats

In [7]:
dfPrices_test = copy.deepcopy(dfPrices)

In [8]:
dfPrices_test.set_index("Unnamed: 0",inplace=True)

In [9]:
dfPrices_test.index.name = None

In [10]:
dfPrices_test

,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
1999-12-31,0.917969,3.806250,36.820244,0.6567,798.154631,1.0062,396.794861,288.00,41.156250,5.3800,146.875000
2000-01-03,0.999442,4.468750,35.498489,0.6585,793.868670,1.0243,384.615387,289.00,43.500000,5.4050,145.437500
2000-01-04,0.915179,4.096875,33.421452,0.6550,796.761677,1.0296,369.230774,282.25,41.468750,5.3250,139.750000
2000-01-05,0.928571,3.487500,33.940708,0.6579,793.118215,1.0321,368.589752,280.00,41.812500,5.1450,140.000000
2000-01-06,0.848214,3.278125,33.043808,0.6533,795.091984,1.0328,373.517639,281.10,39.375000,5.1200,137.750000
...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,152.339996,120.599998,17.690001,0.6394,844.938544,0.9966,73.000000,1653.17,27.410000,19.3473,384.920013
2022-10-26,149.350006,115.660004,18.139999,0.6497,849.013953,1.0081,75.459999,1664.57,27.209999,19.5815,382.019989
2022-10-27,144.800003,110.959999,18.030001,0.6452,853.391436,0.9964,76.000000,1663.31,26.270000,19.6014,379.980011
2022-10-28,155.740005,103.410004,18.480000,0.6411,851.238239,0.9965,78.330002,1644.86,29.070000,19.2595,389.019989


In [11]:
logReturns = np.log(dfPrices_test/dfPrices_test.shift(1))

In [12]:
logReturns

,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,0.085034,0.160464,-0.036558,0.002737,-0.005384,0.017829,-0.031176,0.003466,0.055385,0.004636,-0.009835
2000-01-04,-0.088077,-0.086884,-0.060292,-0.005329,0.003638,0.005161,-0.040822,-0.023633,-0.047821,-0.014912,-0.039891
2000-01-05,0.014527,-0.161039,0.015417,0.004418,-0.004583,0.002425,-0.001738,-0.008004,0.008255,-0.034387,0.001787
2000-01-06,-0.090514,-0.061914,-0.026781,-0.007017,0.002486,0.000678,0.013281,0.003921,-0.060064,-0.004871,-0.016202
...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,0.019153,0.006489,0.010227,0.012907,0.009420,0.009274,-0.004919,0.002053,0.008427,0.005983,0.015842
2022-10-26,-0.019822,-0.041824,0.025120,0.015980,0.004812,0.011473,0.033143,0.006872,-0.007323,0.012032,-0.007563
2022-10-27,-0.030939,-0.041485,-0.006082,-0.006950,0.005143,-0.011674,0.007131,-0.000757,-0.035157,0.001016,-0.005354
2022-10-28,0.072834,-0.070468,0.024652,-0.006375,-0.002526,0.000100,0.030197,-0.011154,0.101279,-0.017597,0.023512


In [13]:
cumReturns = logReturns.cumsum()

In [14]:
cumReturns.iloc[0,:] = logReturns.shape[1] * [0]

In [15]:
cumReturns

,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
1999-12-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2000-01-03,0.085034,0.160464,-0.036558,0.002737,-0.005384,0.017829,-0.031176,0.003466,0.055385,0.004636,-0.009835
2000-01-04,-0.003044,0.073580,-0.096850,-0.002592,-0.001747,0.022990,-0.071998,-0.020167,0.007564,-0.010276,-0.049727
2000-01-05,0.011483,-0.087459,-0.081433,0.001826,-0.006330,0.025415,-0.073735,-0.028171,0.015820,-0.044663,-0.047939
2000-01-06,-0.079031,-0.149373,-0.108214,-0.005191,-0.003845,0.026093,-0.060454,-0.024250,-0.044245,-0.049534,-0.064141
...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,5.111706,3.455835,-0.733048,-0.026697,0.056962,-0.009587,-1.692960,1.747489,-0.406468,1.279865,0.963454
2022-10-26,5.091884,3.414010,-0.707928,-0.010717,0.061773,0.001887,-1.659817,1.754362,-0.413791,1.291897,0.955891
2022-10-27,5.060945,3.372525,-0.714011,-0.017667,0.066916,-0.009787,-1.652686,1.753604,-0.448948,1.292913,0.950537
2022-10-28,5.133780,3.302057,-0.689359,-0.024042,0.064390,-0.009687,-1.622489,1.742450,-0.347669,1.275316,0.974049


In [16]:
logReturns.dropna(inplace=True)

In [17]:
calcSummaryStatistics(logReturns)

,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
Average Annual Return,23.33%,15.01%,-3.20%,-0.12%,0.27%,-0.08%,-7.43%,7.91%,-1.69%,5.79%,4.41%
Annual Risk,42.01%,51.09%,26.73%,12.77%,4.00%,9.78%,34.31%,17.43%,38.35%,30.66%,20.25%
Sharpe Ratio,0.555392,0.293800,-0.119914,-0.009253,0.068470,-0.008414,-0.216461,0.453934,-0.043982,0.188895,0.217698
maxDD,-170.38%,-270.61%,-110.69%,-65.17%,-29.92%,-51.09%,-235.22%,-59.07%,-182.43%,-139.71%,-83.18%
Success Ratio,52.55%,51.41%,51.81%,52.50%,52.25%,50.80%,49.97%,52.73%,51.43%,53.64%,54.18%


In [18]:
percent_function(logReturns.corr(), pct_format='{:.2%}')

,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
AAPL,100.00%,35.94%,26.09%,19.69%,-13.34%,1.03%,34.01%,-1.85%,48.53%,7.46%,53.89%
AMZN,35.94%,100.00%,18.47%,17.44%,-10.06%,1.06%,27.80%,-2.40%,38.62%,4.46%,48.14%
ATT,26.09%,18.47%,100.00%,24.12%,-11.80%,6.89%,39.89%,-1.41%,34.23%,6.05%,56.73%
AUD,19.69%,17.44%,24.12%,100.00%,1.48%,55.01%,28.11%,34.13%,23.52%,41.64%,44.23%
BOND,-13.34%,-10.06%,-11.80%,1.48%,100.00%,14.58%,-18.54%,20.32%,-16.01%,9.76%,-20.68%
EUR,1.03%,1.06%,6.89%,55.01%,14.58%,100.00%,7.02%,38.27%,0.25%,34.98%,11.64%
GE,34.01%,27.80%,39.89%,28.11%,-18.54%,7.02%,100.00%,-4.70%,41.82%,6.54%,67.10%
GOLD,-1.85%,-2.40%,-1.41%,34.13%,20.32%,38.27%,-4.70%,100.00%,-2.05%,76.59%,0.17%
INTC,48.53%,38.62%,34.23%,23.52%,-16.01%,0.25%,41.82%,-2.05%,100.00%,7.40%,65.06%
SILVER,7.46%,4.46%,6.05%,41.64%,9.76%,34.98%,6.54%,76.59%,7.40%,100.00%,16.70%


# Recreating Strategy:
- We aim to find strategy for the 8 instruments we have choosen based on previous analysis
- **AAPL, AMZN, ATT, GE, INTC, BOND, GOLD, EUR**


In [19]:
def movingAverageCrossover(dataFrame,fastWindow =42,slowWindow=260,flatOrShort=0,colname="EUR"):
    dataFrame.loc[:,'FastSMA'] = dataFrame[colname].rolling(fastWindow).mean()
    dataFrame.loc[:,'SlowSMA'] = dataFrame[colname].rolling(slowWindow).mean()
    dataFrame.dropna(inplace=True)
    
#     ax = dataFrame.plot(figsize=(10,5), title='SPY Close Price Overlaid with Fast and Slow MA', grid=True)
#     ax.yaxis.set_major_formatter('${x:1,.0f}')
    
    
    # SMA strategy: if rising trend, stay long, otherwise go flat depending on value of flag flatOrShort
    dataFrame.loc[:,'Position'] = np.where(dataFrame['FastSMA'] > dataFrame['SlowSMA'], 1, flatOrShort)
    dataFrame.loc[:,'OriginalReturns_Benchmark'] = np.log(dataFrame[colname] / dataFrame[colname].shift(1))
    dataFrame.loc[:,'SMAReturns'] = dataFrame['Position'].shift(1) * dataFrame['OriginalReturns_Benchmark']
    
    dataFrame_ret = dataFrame[["OriginalReturns_Benchmark","SMAReturns"]]
    
#     ax = dataFrame_ret[['OriginalReturns_Benchmark', 'SMAReturns']].cumsum().plot(figsize=(10,5), ylabel='NAV', title='SPY Original and SMA Strategy Cumulative NAVs', grid=True)
#     ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0, decimals=0))
    return dataFrame_ret

In [20]:
def bollingerBands(dataFrame,lookbackWindow=20,stdevBand=2,colname="EUR"):
    dataFrame.loc[:,'Mean'] = dataFrame[colname].rolling(lookbackWindow).mean()
    dataFrame.loc[:,'Stdev'] = dataFrame[colname].rolling(lookbackWindow).std()
    dataFrame.loc[:,'Upper'] = dataFrame['Mean'] + stdevBand * dataFrame['Stdev']
    dataFrame.loc[:,'Lower'] = dataFrame['Mean'] - stdevBand * dataFrame['Stdev']
    dataFrame.dropna(inplace=True)
    
#     ax = dataFrame[['SPY', 'Mean', 'Upper', 'Lower']].plot(figsize=aspect_ratio, title='SPY Price with Bollinger Bands', ylabel='Price', grid=True)
#     ax.yaxis.set_major_formatter('${x:1,.0f}')
    
    date0 = dataFrame.index[0]
    if dataFrame.loc[date0,colname] >= dataFrame.loc[date0, 'Upper']:
        dataFrame.loc[date0, 'Position'] = -1
    elif dataFrame.loc[date0,'EUR'] <= dataFrame.loc[date0,'Lower']:
        dataFrame.loc[date0, 'Position'] = 1
    else:
        dataFrame.loc[date0, 'Position'] = 0
    
    for i in range(1, dataFrame.shape[0]):
        today=dataFrame.index[i]
        yesterday=dataFrame.index[i-1]
        if dataFrame.loc[today,colname] >= dataFrame.loc[today,'Upper']: # if close is above upper
            dataFrame.loc[today,'Position'] = -1 # then go short
        elif dataFrame.loc[today,colname] <= dataFrame.loc[today,'Lower']: # if close is below lower
            dataFrame.loc[today,'Position'] = 1 # then go long
        elif dataFrame.loc[yesterday,'Position'] == -1 and dataFrame.loc[today,colname] <= dataFrame.loc[today,'Mean']: # if prev day is short and we're now below the mean
            dataFrame.loc[today,'Position']=0 # then flatten
        elif dataFrame.loc[yesterday,'Position'] == 1 and dataFrame.loc[today, colname] >= dataFrame.loc[today, 'Mean']: # conversely...
            dataFrame.loc[today,'Position']=0 # then also flatten
        else: # otherwise just hold yesterday's position
            dataFrame.loc[today,'Position']=dataFrame.loc[yesterday,'Position']
            
    dataFrame.loc[:,'OriginalReturns_Benchmark'] = np.log(dataFrame[colname] / dataFrame[colname].shift(1))
    dataFrame.loc[:,'BBReturns'] = dataFrame['Position'].shift(1) * dataFrame['OriginalReturns_Benchmark']
    dataFrame.dropna(inplace=True)
    return dataFrame[['OriginalReturns_Benchmark', 'BBReturns']]

## Strategy MA going Flat for GOLD:

In [21]:
new_df = copy.deepcopy(dfPrices_test)

In [22]:
new_df_ma_gold = movingAverageCrossover(new_df.copy(),fastWindow=33,slowWindow=220,colname="GOLD")

In [23]:
new_df_ma_gold

,OriginalReturns_Benchmark,SMAReturns
2000-11-13,NaN,NaN
2000-11-14,0.001134,0.0
2000-11-15,0.001322,0.0
2000-11-16,0.005644,0.0
2000-11-17,-0.002818,-0.0
...,...,...
2022-10-25,0.002053,0.0
2022-10-26,0.006872,0.0
2022-10-27,-0.000757,-0.0
2022-10-28,-0.011154,-0.0


In [24]:
new_df.shape

(5704, 11)

In [25]:
calcSummaryStatistics(new_df_ma_gold.dropna())

,OriginalReturns_Benchmark,SMAReturns
Average Annual Return,8.63%,8.14%
Annual Risk,17.50%,14.24%
Sharpe Ratio,0.493339,0.571874
maxDD,-59.07%,-40.43%
Success Ratio,53.21%,68.16%


## Strategy MA going Flat for Amazon:

In [26]:
new_df_ma_AMZN = movingAverageCrossover(new_df.copy(),fastWindow=33,slowWindow=220,colname="AMZN")

In [27]:
new_df.shape

(5704, 11)

In [28]:
new_df_ma_AMZN

,OriginalReturns_Benchmark,SMAReturns
2000-11-13,NaN,NaN
2000-11-14,0.039653,0.0
2000-11-15,0.019252,0.0
2000-11-16,-0.058905,-0.0
2000-11-17,-0.013575,-0.0
...,...,...
2022-10-25,0.006489,0.0
2022-10-26,-0.041824,-0.0
2022-10-27,-0.041485,-0.0
2022-10-28,-0.070468,-0.0


In [29]:
calcSummaryStatistics(new_df_ma_AMZN.dropna())

,OriginalReturns_Benchmark,SMAReturns
Average Annual Return,20.38%,18.58%
Annual Risk,48.06%,32.10%
Sharpe Ratio,0.424177,0.578937
maxDD,-159.76%,-105.04%
Success Ratio,51.71%,67.18%


## Strategy MA going Flat for AAPL:

In [30]:
new_df_ma_AAPL = movingAverageCrossover(new_df.copy(),fastWindow=33,slowWindow=220,colname="AAPL")

In [31]:
new_df_ma_AAPL

,OriginalReturns_Benchmark,SMAReturns
2000-11-13,NaN,NaN
2000-11-14,0.044171,0.0
2000-11-15,-0.018691,-0.0
2000-11-16,-0.045024,-0.0
2000-11-17,-0.026670,-0.0
...,...,...
2022-10-25,0.019153,0.0
2022-10-26,-0.019822,-0.0
2022-10-27,-0.030939,-0.0
2022-10-28,0.072834,0.0


In [32]:
new_df.shape

(5704, 11)

In [33]:
calcSummaryStatistics(new_df_ma_AAPL.dropna())

,OriginalReturns_Benchmark,SMAReturns
Average Annual Return,28.89%,23.67%
Annual Risk,37.04%,28.79%
Sharpe Ratio,0.780100,0.822146
maxDD,-93.82%,-85.28%
Success Ratio,52.63%,64.75%


## Strategy Bolinger Band for ATT:

In [34]:
new_df_bb_ATT = bollingerBands(new_df.copy(),lookbackWindow=8,stdevBand=1,colname="ATT")

In [35]:
new_df_bb_ATT

,OriginalReturns_Benchmark,BBReturns
2000-01-12,-0.053831,-0.053831
2000-01-13,-0.024207,-0.024207
2000-01-14,0.028681,0.028681
2000-01-18,-0.021053,-0.021053
2000-01-19,-0.013772,-0.013772
...,...,...
2022-10-25,0.010227,-0.010227
2022-10-26,0.025120,-0.025120
2022-10-27,-0.006082,0.006082
2022-10-28,0.024652,-0.024652


In [36]:
new_df.shape

(5704, 11)

In [37]:
calcSummaryStatistics(new_df_bb_ATT.dropna())

,OriginalReturns_Benchmark,BBReturns
Average Annual Return,-2.75%,8.44%
Annual Risk,26.69%,23.50%
Sharpe Ratio,-0.103184,0.359219
maxDD,-110.69%,-49.42%
Success Ratio,51.83%,63.27%


## Strategy Bolinger Band for BOND:

In [38]:
new_df_bb_BOND = bollingerBands(new_df.copy(),lookbackWindow=8,stdevBand=1,colname="BOND")

In [39]:
new_df_bb_BOND

,OriginalReturns_Benchmark,BBReturns
2000-01-12,-0.001434,-0.001434
2000-01-13,0.004371,0.004371
2000-01-14,-0.001743,-0.001743
2000-01-18,-0.002567,-0.002567
2000-01-19,0.001211,0.001211
...,...,...
2022-10-25,0.009420,0.009420
2022-10-26,0.004812,0.000000
2022-10-27,0.005143,-0.005143
2022-10-28,-0.002526,0.002526


In [40]:
new_df.shape

(5704, 11)

In [41]:
calcSummaryStatistics(new_df_bb_BOND.dropna())

,OriginalReturns_Benchmark,BBReturns
Average Annual Return,0.31%,0.42%
Annual Risk,4.00%,3.55%
Sharpe Ratio,0.077496,0.117248
maxDD,-29.92%,-19.43%
Success Ratio,52.26%,61.96%


## Strategy MA going short for GE:

In [42]:
new_df_ma_GE = movingAverageCrossover(new_df.copy(),fastWindow=33,slowWindow=220,colname="GE",flatOrShort=-1)

In [43]:
new_df_ma_GE

,OriginalReturns_Benchmark,SMAReturns
2000-11-13,NaN,NaN
2000-11-14,0.025196,0.025196
2000-11-15,-0.004751,-0.004751
2000-11-16,0.002378,0.002378
2000-11-17,-0.014354,-0.014354
...,...,...
2022-10-25,-0.004919,0.004919
2022-10-26,0.033143,-0.033143
2022-10-27,0.007131,-0.007131
2022-10-28,0.030197,-0.030197


In [44]:
new_df.shape

(5704, 11)

In [45]:
calcSummaryStatistics(new_df_ma_GE.dropna())

,OriginalReturns_Benchmark,SMAReturns
Average Annual Return,-7.71%,5.39%
Annual Risk,34.16%,34.16%
Sharpe Ratio,-0.225738,0.157766
maxDD,-227.09%,-190.48%
Success Ratio,49.96%,50.89%


## Strategy MA going flat for EUR:

In [46]:
new_df_ma_EUR = movingAverageCrossover(new_df.copy(),fastWindow=33,slowWindow=220,colname="EUR")

In [47]:
new_df_ma_EUR

,OriginalReturns_Benchmark,SMAReturns
2000-11-13,NaN,NaN
2000-11-14,0.000583,0.0
2000-11-15,-0.000117,-0.0
2000-11-16,-0.005027,-0.0
2000-11-17,-0.006349,-0.0
...,...,...
2022-10-25,0.009274,0.0
2022-10-26,0.011473,0.0
2022-10-27,-0.011674,-0.0
2022-10-28,0.000100,0.0


In [48]:
calcSummaryStatistics(new_df_ma_EUR.dropna())

,OriginalReturns_Benchmark,SMAReturns
Average Annual Return,0.67%,1.24%
Annual Risk,9.66%,6.65%
Sharpe Ratio,0.069843,0.186926
maxDD,-51.09%,-21.95%
Success Ratio,51.04%,73.76%


## Strategy Bollinger Bands for INTC:

In [49]:
new_df_bb_INTC = bollingerBands(new_df.copy(),lookbackWindow=8,stdevBand=1,colname="INTC")

In [50]:
new_df_bb_INTC

,OriginalReturns_Benchmark,BBReturns
2000-01-12,0.017272,-0.017272
2000-01-13,-0.002057,0.002057
2000-01-14,0.123790,-0.123790
2000-01-18,-0.009138,0.009138
2000-01-19,-0.020403,0.020403
...,...,...
2022-10-25,0.008427,-0.008427
2022-10-26,-0.007323,0.007323
2022-10-27,-0.035157,0.035157
2022-10-28,0.101279,0.000000


In [51]:
calcSummaryStatistics(new_df_bb_INTC.dropna())

,OriginalReturns_Benchmark,BBReturns
Average Annual Return,-2.08%,15.62%
Annual Risk,38.28%,32.81%
Sharpe Ratio,-0.054341,0.476156
maxDD,-182.43%,-60.91%
Success Ratio,51.40%,62.38%


# Combine all data and clean:

In [52]:
new_df_ma_gold.head()

,OriginalReturns_Benchmark,SMAReturns
2000-11-13,NaN,NaN
2000-11-14,0.001134,0.0
2000-11-15,0.001322,0.0
2000-11-16,0.005644,0.0
2000-11-17,-0.002818,-0.0


In [53]:
new_df_ma_AMZN.head()

,OriginalReturns_Benchmark,SMAReturns
2000-11-13,NaN,NaN
2000-11-14,0.039653,0.0
2000-11-15,0.019252,0.0
2000-11-16,-0.058905,-0.0
2000-11-17,-0.013575,-0.0


In [54]:
new_df_ma_AMZN.tail()

,OriginalReturns_Benchmark,SMAReturns
2022-10-25,0.006489,0.0
2022-10-26,-0.041824,-0.0
2022-10-27,-0.041485,-0.0
2022-10-28,-0.070468,-0.0
2022-10-31,-0.009424,-0.0


In [55]:
new_df_ma_AAPL.head()

,OriginalReturns_Benchmark,SMAReturns
2000-11-13,NaN,NaN
2000-11-14,0.044171,0.0
2000-11-15,-0.018691,-0.0
2000-11-16,-0.045024,-0.0
2000-11-17,-0.026670,-0.0


In [56]:
new_df_bb_ATT.head()

,OriginalReturns_Benchmark,BBReturns
2000-01-12,-0.053831,-0.053831
2000-01-13,-0.024207,-0.024207
2000-01-14,0.028681,0.028681
2000-01-18,-0.021053,-0.021053
2000-01-19,-0.013772,-0.013772


In [57]:
new_df_ma_GE.head()

,OriginalReturns_Benchmark,SMAReturns
2000-11-13,NaN,NaN
2000-11-14,0.025196,0.025196
2000-11-15,-0.004751,-0.004751
2000-11-16,0.002378,0.002378
2000-11-17,-0.014354,-0.014354


In [58]:
new_df_bb_INTC.head()

,OriginalReturns_Benchmark,BBReturns
2000-01-12,0.017272,-0.017272
2000-01-13,-0.002057,0.002057
2000-01-14,0.123790,-0.123790
2000-01-18,-0.009138,0.009138
2000-01-19,-0.020403,0.020403


In [59]:
new_df_bb_BOND.head()

,OriginalReturns_Benchmark,BBReturns
2000-01-12,-0.001434,-0.001434
2000-01-13,0.004371,0.004371
2000-01-14,-0.001743,-0.001743
2000-01-18,-0.002567,-0.002567
2000-01-19,0.001211,0.001211


In [60]:
new_df_ma_EUR.head()

,OriginalReturns_Benchmark,SMAReturns
2000-11-13,NaN,NaN
2000-11-14,0.000583,0.0
2000-11-15,-0.000117,-0.0
2000-11-16,-0.005027,-0.0
2000-11-17,-0.006349,-0.0


In [61]:
new_df_ma_AMZN = new_df_ma_AMZN.rename(columns = {'SMAReturns':'AMZN'})
new_df_ma_AAPL= new_df_ma_AAPL.rename(columns = {'SMAReturns':'AAPL'})
new_df_ma_gold = new_df_ma_gold.rename(columns = {'SMAReturns':'GOLD'})
new_df_ma_GE = new_df_ma_GE.rename(columns = {'SMAReturns':'GE'})
new_df_ma_EUR = new_df_ma_EUR.rename(columns = {'SMAReturns':'EUR'})
new_df_bb_ATT = new_df_bb_ATT.rename(columns = {'BBReturns':'ATT'})
new_df_bb_INTC = new_df_bb_INTC.rename(columns = {'BBReturns':'INTC'})
new_df_bb_BOND = new_df_bb_BOND.rename(columns = {'BBReturns':'BOND'})

In [62]:
new_df_ma_AMZN = new_df_ma_AMZN[["AMZN"]]
new_df_ma_AAPL = new_df_ma_AAPL[["AAPL"]]
new_df_ma_gold = new_df_ma_gold[["GOLD"]]
new_df_ma_GE = new_df_ma_GE[["GE"]]
new_df_ma_EUR = new_df_ma_EUR[["EUR"]]
new_df_bb_ATT = new_df_bb_ATT[["ATT"]]
new_df_bb_INTC = new_df_bb_INTC[["INTC"]]
new_df_bb_BOND = new_df_bb_BOND[["BOND"]]

In [63]:
new_df_ma_gold

,GOLD
2000-11-13,NaN
2000-11-14,0.0
2000-11-15,0.0
2000-11-16,0.0
2000-11-17,-0.0
...,...
2022-10-25,0.0
2022-10-26,0.0
2022-10-27,-0.0
2022-10-28,-0.0


# Q3: Reasons for choosing the strategies for each instrument

- For each instrument we worked on getting the best Sharpe ratio individually.
- We've eliminated instruments that were highly correlated to others.

### For Gold, Amazon, Apple and EUR currency
- We got the best Sharpe ratios for these instruments by using the MA crossover strategy going "flat".
- We iterated through the process of finding the best fast and slow windows for implementing these strategies.
- We've chosen the fast window of 33 and slow window of 220 for this strategy as they provided the best Sharpe ratio

### For GE
- We got the best Sharpe ratios for these instruments by using the MA crossover strategy going "short".
- We've chosen the fast window of 33 and slow window of 220 for this strategy.

### For ATT, INTC and BONDS
- We got the best Sharpe ratio for these instruments by using Bollinger Bands.
- We iterated through the strategy to look for best lookback window and the std dev from 2 to 50 lookback windows and 1 to 4 std deviations.
- ATT had the best Sharpe ratio when lookback window was 9 whereas for INTC we found out 8 as the best lookback window.
- On implementing lookback window as 8 for ATT the Sharpe ratio didn't change much as it did for implementing a lookback window of 9 for INTC.
- We finalised on going with a lookback window of 8 and std dev of 1.


In [64]:
dfMerge = new_df_ma_gold.join([new_df_ma_AMZN,new_df_ma_AAPL,new_df_ma_GE,new_df_ma_EUR,new_df_bb_ATT, \
                               new_df_bb_INTC,new_df_bb_BOND], \
                              how ='inner')

In [65]:
dfMerge.dropna(inplace=True)

In [66]:
dfMerge['GOLD'].value_counts()

 0.000000    1682
 0.002831       2
-0.000235       2
-0.000717       2
-0.001951       2
             ... 
-0.003918       1
 0.000491       1
 0.025187       1
-0.003834       1
-0.001254       1
Name: GOLD, Length: 3799, dtype: int64

In [67]:
calcSummaryStatistics(dfMerge.dropna())

,GOLD,AMZN,AAPL,GE,EUR,ATT,INTC,BOND
Average Annual Return,8.14%,18.58%,23.67%,5.39%,1.24%,7.96%,16.95%,0.45%
Annual Risk,14.24%,32.10%,28.79%,34.16%,6.65%,22.77%,31.77%,3.56%
Sharpe Ratio,0.571874,0.578937,0.822146,0.157766,0.186926,0.349487,0.533476,0.125225
maxDD,-40.43%,-105.04%,-85.28%,-190.48%,-21.95%,-49.42%,-60.40%,-19.43%
Success Ratio,68.16%,67.18%,64.75%,50.89%,73.76%,63.35%,62.51%,62.02%


# Portfolio Weights:

In [68]:
weights_from_PartA = [0.21275226, 0.0655385 , 0.20547114, 0.1136603 , 0.01661343,0.11044183, 0.03671583, 0.23880671]
weights_eq = np.array([0.125]*8)

In [69]:
bestport = copy.deepcopy(dfMerge)
eqport = copy.deepcopy(dfMerge)

In [71]:
bestport['Best Port'] = (weights_from_PartA * bestport).sum(axis=1) 

In [74]:
eqport = copy.deepcopy(dfMerge)

In [77]:
bestport['Port'] = (weights_eq * eqport).sum(axis=1)

In [85]:
bestport.dropna(inplace=True)

In [86]:
bestport.loc['2019-01-01':'2022-10-31']

,GOLD,AMZN,AAPL,GE,EUR,ATT,INTC,BOND,Best Port,Port
2019-01-02,0.0,0.0,0.0,-0.061479,-0.0,0.000000,-0.003191,-0.001133,-0.007376,-0.008225
2019-01-03,0.0,-0.0,-0.0,-0.001241,0.0,-0.001353,0.056584,-0.005373,0.000504,0.006077
2019-01-04,-0.0,0.0,0.0,-0.020872,0.0,-0.025368,0.000000,0.004796,-0.004029,-0.005181
2019-01-07,0.0,0.0,-0.0,-0.060124,0.0,-0.017966,0.000000,0.000424,-0.008717,-0.009708
2019-01-08,-0.0,0.0,0.0,0.020810,-0.0,-0.012546,0.000000,0.001391,0.001312,0.001207
...,...,...,...,...,...,...,...,...,...,...
2022-10-25,0.0,0.0,0.0,0.004919,0.0,-0.010227,-0.008427,0.009420,0.001370,-0.000539
2022-10-26,0.0,-0.0,-0.0,-0.033143,0.0,-0.025120,0.007323,0.000000,-0.006272,-0.006367
2022-10-27,-0.0,-0.0,-0.0,-0.007131,-0.0,0.006082,0.035157,-0.005143,-0.000076,0.003621
2022-10-28,-0.0,-0.0,0.0,-0.030197,0.0,-0.024652,0.000000,0.002526,-0.005552,-0.006540


In [87]:
calcSummaryStatistics(bestport)

,GOLD,AMZN,AAPL,GE,EUR,ATT,INTC,BOND,Best Port,Port
Average Annual Return,8.14%,18.58%,23.67%,5.39%,1.24%,7.96%,16.95%,0.45%,10.06%,10.30%
Annual Risk,14.24%,32.10%,28.79%,34.16%,6.65%,22.77%,31.77%,3.56%,9.41%,9.93%
Sharpe Ratio,0.571874,0.578937,0.822146,0.157766,0.186926,0.349487,0.533476,0.125225,1.068722,1.037221
maxDD,-40.43%,-105.04%,-85.28%,-190.48%,-21.95%,-49.42%,-60.40%,-19.43%,-20.03%,-21.77%
Success Ratio,68.16%,67.18%,64.75%,50.89%,73.76%,63.35%,62.51%,62.02%,54.49%,54.10%


In [88]:
calcSummaryStatistics(bestport.loc['2019-01-01':'2022-10-31'])

,GOLD,AMZN,AAPL,GE,EUR,ATT,INTC,BOND,Best Port,Port
Average Annual Return,7.80%,12.33%,23.99%,-39.78%,1.27%,-0.32%,25.80%,-4.36%,2.77%,3.34%
Annual Risk,13.66%,25.11%,32.00%,45.93%,3.29%,24.61%,33.52%,4.65%,11.79%,12.20%
Sharpe Ratio,0.570678,0.491197,0.749593,-0.866071,0.387323,-0.013125,0.769741,-0.937091,0.234773,0.273882
maxDD,-21.00%,-25.81%,-37.73%,-172.27%,-5.08%,-39.16%,-55.31%,-17.44%,-20.03%,-20.26%
Success Ratio,67.67%,68.40%,61.94%,47.24%,86.44%,61.94%,62.98%,61.31%,52.45%,51.93%


We observe the below
- When comparing the total shapre the best portfolio slightly outperforms the equal weighted one
- For the out of sampled window the sharpe is much worse now for the weights created before compared to equal weighted portfolio
- For the out of sampled window the sharpe is also much lower than most of the individual instruments 

**Our Failure can be attribute towards the fixed weighted portfolio that we have created which in this cased has led to a worse shape as the instruments now have drastically changed especially BOND which now has NEGATIVE sharpe**

"We pledge on our honor that we have not given nor received any unauthorized assistance on 
this assignment with other groups." 
- Abhivrudh Ramesh, Arjan Singh, Atharva Patil, Kunal Jain, Uzair Israrahmed.   